In [9]:
import numpy as np
import qiskit_nature
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit_nature.second_q.transformers import FreezeCoreTransformer
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock

qiskit_nature.settings.use_pauli_sum_op = False
from qiskit_nature.second_q.drivers import PySCFDriver
import matplotlib.pyplot as plt
from qiskit.circuit.library import EfficientSU2
from qiskit_aer.primitives import Estimator
from qiskit.algorithms.optimizers import SLSQP,SPSA,CG


In [15]:
def get_qubit_info(dist, Molecule):


    #Select Molecule
    if Molecule=="1":
        molecule = MoleculeInfo(
        symbols=["C","C"],
        coords=([0.0, 0.0, 0.0], [dist,0.0,0.0]),
        multiplicity=1,  
        charge=0,
        )
    elif Molecule=="2":
        molecule = MoleculeInfo(
        symbols=["S","Mo", "S"],
        coords=([0.0, 0.0, 0.0], [dist,0.0,0.0],[2*dist, 0.0, 0.0]),
        multiplicity=1, 
        charge=0,
        )
    elif Molecule=="3":
        molecule = MoleculeInfo(
        symbols=["P","P"],
        coords=([0.0, 0.0, 0.0], [dist,0.0,0.0]),
        multiplicity=1,
        charge=0,
        )
    elif Molecule=="4":
        molecule = MoleculeInfo(
        symbols=["Si","Si"],
        coords=([0.0, 0.0, 0.0], [dist,0.0,0.0]),
        multiplicity=1,
        charge=0,
        )

    driver = PySCFDriver.from_molecule(molecule,basis='sto3g')

    #  properties of the Molecule Selected
    properties_molecule = driver.run()
    prop = FreezeCoreTransformer(
        freeze_core=True
    ).transform(properties_molecule)

    no_particles = prop.num_particles
    no_spatial_orbitals = prop.num_spatial_orbitals

    mapper = ParityMapper(num_particles=no_particles)
    qubit = mapper.map(prop.second_q_ops()[0])
    return qubit, no_particles, no_spatial_orbitals, prop, mapper

In [ ]:

print("Select the anode molecule to find the minimum energy")
print("1. Graphene  \n2. MoS2 \n3. Phosphorene \n4. Silicene ")
molecule=input("Selected Molecule")
print(molecule)
bond_lengths = np.arange(1.0, 3.0, 0.25)

vqe_E = []
#optimizer = SLSQP(maxiter=5)
optimizer=CG(maxiter=5)
estimator = Estimator(approximation=True)
for bl in bond_lengths:
    print("\nCalculation Started - Step ",bl)
    (qubit, no_particles, no_spatial_orbitals, prop, mapper) = get_qubit_info(
        bl,molecule)
    print("qubit Info Obtained")
    initial_state = HartreeFock(no_spatial_orbitals, no_particles, mapper)
    print("Initial State configured")
    vari_form = UCCSD(
        no_spatial_orbitals, no_particles, mapper, initial_state=initial_state
    )
    
    vqe = VQE(
        estimator,
        vari_form,
        optimizer,
        initial_point=[0] * vari_form.num_parameters,
    )
    calc = vqe.compute_minimum_eigenvalue(qubit)
    result = problem.interpret(calc).total_energies[0].real
    vqe_E.append(result)
    print(
        f"Interatomic Distance: {np.round(bl, 2)}",
        f"VQE Energy: {result:.5f}",
     
    )


plt.plot(bond_lengths, vqe_E, label="VQE Estimated Energy")
plt.xlabel("Atomic distance (Angstrom)")
plt.ylabel("Energy (Hartree)")
plt.legend()
plt.show()

Select the anode molecule to find the minimum energy
1. Graphene  
2. MoS2 
3. Phosphorene 
4. Silicene 


Selected Molecule 1


1

Calculation Started - Step  1.0
qubit Info Obtained
Initial State configured
